##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Context Caching Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Caching.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook introduces context caching with the Gemini API and provides examples of interacting with the Apollo 11 transcript using the Python SDK. For a more comprehensive look, check out [the caching guide](https://ai.google.dev/gemini-api/docs/caching?lang=python).

In [ ]:
!pip install -qU 'google-generativeai>=0.7.0'

In [1]:
import google.generativeai as genai
from google.generativeai import caching

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [2]:
from google.colab import userdata

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## Upload a file

A common pattern with the Gemini API is to ask a number of questions of the same document. Context caching is designed to assist with this case, and can be more efficient by avoiding the need to pass the same tokens through the model for each new request.

Start by uploading a large file with the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb).

In [3]:
!wget -q https://storage.googleapis.com/generativeai-downloads/data/a11.txt
!head a11.txt

INTRODUCTION

This is the transcription of the Technical Air-to-Ground Voice Transmission (GOSS NET 1) from the Apollo 11 mission.

Communicators in the text may be identified according to the following list.

Spacecraft:
CDR	Commander	Neil A. Armstrong
CMP	Command module pilot   	Michael Collins
LMP	Lunar module pilot	Edwin E. ALdrin, Jr.


In [7]:
# To reduce the cost of experiment. I want to use only 1/10 of the file.

with open('a11.txt', 'r') as file:
  text = file.read()
tenth_of_text = text[:len(text)//10]

with open('a12.txt', 'w') as file:
  file.write(tenth_of_text)


In [8]:
document = genai.upload_file(path="a12.txt")

## Cache the prompt

Next create a [`CachedContent`](https://ai.google.dev/api/python/google/generativeai/protos/CachedContent) object specifying the prompt you want to use, including the file and other fields you wish to cache. In this example the `system_instruction` has been set, and the document was provided in the prompt.

In [9]:
# Note that caching requires a frozen model, e.g. one with a `-001` version suffix.
model_name = "gemini-1.5-flash-001"

apollo_cache = caching.CachedContent.create(
    model=model_name,
    system_instruction="You are an expert at analyzing transcripts.",
    contents=[document],
)

apollo_cache

CachedContent(
    name='cachedContents/c1tp9dntx6cs',
    model='models/gemini-1.5-flash-001',
    display_name='',
    usage_metadata={
        'total_token_count': 33731,
    },
    create_time=2024-08-04 09:58:24.234303+00:00,
    update_time=2024-08-04 09:58:24.234303+00:00,
    expire_time=2024-08-04 10:58:23.465918+00:00
)

## Manage the cache expiry

Once you have a `CachedContent` object, you can update the expiry time to keep it alive while you need it.

In [10]:
import datetime

apollo_cache.update(ttl=datetime.timedelta(hours=2))
apollo_cache

CachedContent(
    name='cachedContents/c1tp9dntx6cs',
    model='models/gemini-1.5-flash-001',
    display_name='',
    usage_metadata={
        'total_token_count': 33731,
    },
    create_time=2024-08-04 09:58:24.234303+00:00,
    update_time=2024-08-04 09:58:36.094840+00:00,
    expire_time=2024-08-04 11:58:36.074026+00:00
)

## Use the cache for generation

As the `CachedContent` object refers to a specific model and parameters, you must create a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) using [`from_cached_content`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#from_cached_content). Then, generate content as you would with a directly instantiated model object.

In [11]:
apollo_model = genai.GenerativeModel.from_cached_content(cached_content=apollo_cache)

response = apollo_model.generate_content("Find a lighthearted moment from this transcript")
print(response.text)

A lighthearted moment occurs on page 11 of the transcript, around 01:42:45:

**CMP:**  "Houston, Apollo 11. We're standing by for a GO for sequence logic ON."
**CC:** "Apollo 11, this is Houston. Go ahead and we'll watch you on TM."
**CDR:** "Okay. Sequence logic, two of them. Sequence logic 1 and 2 coming up and ON." 

This brief exchange shows the astronauts' playful use of language ("coming up" and "ON") in contrast to the serious nature of their mission. It reflects the astronauts' ability to maintain a sense of humor even in the midst of a stressful situation. 



You can inspect token usage through `usage_metadata`. Note that the cached prompt tokens are included in `prompt_token_count`, but excluded from the `total_token_count`.

In [12]:
response.usage_metadata

prompt_token_count: 33740
candidates_token_count: 154
total_token_count: 33894
cached_content_token_count: 33731

You can ask new questions of the model, and the cache is reused.

In [13]:
chat = apollo_model.start_chat()
response = chat.send_message("Give me a quote from the most important part of the transcript.")
print(response.text)

StopCandidateException: finish_reason: SAFETY
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: MEDIUM
}


In [14]:
chat = apollo_model.start_chat()
response = chat.send_message("Give me a quote from the most important part of the transcript.",
safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL' : 'BLOCK_NONE',
        'DANGEROUS' : 'BLOCK_NONE'
    }
                             )
print(response.text)

The most important part of the transcript is the section detailing the Translunar Injection (TLI) burn, which marked the beginning of Apollo 11's journey to the Moon. 

Here's a quote from that section: 

**"00 02 26 38 CC
Apollo 11, this is Houston. You are GO for TLI. Over."**

This quote signifies the moment Mission Control gave the crew the green light to initiate the TLI burn, propelling them on their historic journey to the lunar surface. 



In [15]:
response = chat.send_message("What was recounted after that?")
print(response.text)

After Mission Control gave the "GO" for TLI, the transcript records the following:

* **Confirmation from the crew:**  
   "00 02 26 45 CMP
   Apollo 11. Thank you."

* **Confirmation from Mission Control:** 
   "00 02 26 48 CC
   Roger. Out."

* **Communication about ARIA coverage:**
    "00 02 30 11 CC
   Apollo 11, this is Houston. Over."
    "00 02 30 16 CDR
   Houston, 11."
    "00 02 30 18 CC
   Roger. We'll be coming within range of the ARIA aircraft coverage, here, in about 1 minute. They're going to try uplinking both on S-band and on VHF this time. So if you turn your - make sure your S-band volume is turned up, we'd appreciate it. And we believe that we'll have continuous coverage from now on through the TLI burn. Over."
    "00 02 30 42 CDR
   Very good." 

* **The start of the TLI burn:**
    "00 02 44 19 CMP
   Ignition." 
    "00 02 44 27 CC
   We confirm ignition, and the thrust is GO."

* **Updates during the TLI burn:**
    "00 02 45 14 CC
   Apollo 11, this is Housto

In [16]:
response.usage_metadata

prompt_token_count: 33872
candidates_token_count: 1033
total_token_count: 34905
cached_content_token_count: 33731

## Counting tokens

The `GenerativeModel` object can be used to count the tokens of a request in the same manner as a direct, uncached, model.

In [17]:
apollo_model.count_tokens("How many people are involved in this transcript?")

total_tokens: 9

## Delete the cache

A cache object will be deleted automatically when it expires. You can also explicitly delete a cache object.

In [18]:
print(apollo_cache.name)
apollo_cache.delete()

cachedContents/c1tp9dntx6cs
